In [5]:
import copy
# define rooms and items

# prison cell furniture
bunk_bed = {
    "name": "bunk bed",
    "type": "furniture",
}

sink = {
    "name": "sink",
    "type": "furniture",
}

toilet = {
    "name": "toilet",
    "type": "furniture",
}


# hallway furniture
emergency_box = {
    "name": "emergency box",
    "type": "furniture",
}


# kitchen furniture
fridge = {
    "name": "fridge",
    "type": "furniture",
}

shelf = {
    "name": "shelf",
    "type": "furniture",
}

cooker = {
    "name": "cooker",
    "type": "furniture",
}


# patio furniture
waste_container = {
    "name": "waste container",
    "type": "furniture",
}


#doors
door_a = {
    "name": "cell door",
    "type": "door",
}

door_b = {
    "name": "kitchen door",
    "type": "door",
}

door_c = {
    "name": "air duct",
    "type": "door",
}

door_d = {
    "name": "patio wall",
    "type": "door",
}

#Keys

key_a = {
    "name": "a hair pin", #for cell door
    "type": "key",
    "target": door_a
}

key_b = {
    "name": "an axe", #for kitchen door
    "type": "key",
    "target": door_b
}

key_c = {
    "name": "a knife", # for air duct
    "type": "key",
    "target": door_c
}

key_d = {
    "name": "a rope", # for patio wall
    "type": "key",
    "target": door_d
}


#ROOMS
prison_cell = {
    "name": "prison cell",
    "type": "room",
}

hallway  = {
    "name": "hallway",
    "type": "room",
}

kitchen = {
    "name": "kitchen",
    "type": "room",
}

patio = {
    "name": "patio",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [prison_cell, hallway, kitchen, patio, outside]

all_doors = [door_a, door_b, door_c, door_d]


# define which items/rooms are related

object_relations = {
    "prison cell": [bunk_bed, sink, toilet, door_a],
    "toilet": [key_a],
    "cell door": [prison_cell, hallway],
    
    "hallway": [emergency_box, door_a, door_b, door_c],
    "emergency box": [key_b],
    "kitchen door": [hallway, kitchen], # CHECK THIS kitchen wrong
    "air duct": [hallway, patio], # FIXED NAME AND PUT AIR DUCT
    
    "kitchen": [fridge, cooker, shelf, door_b],
    "fridge": [key_c],
    #door is done as key:door b
    "patio": [waste_container, door_c, door_d],
    "waste container": [key_d],
    "patio wall": [patio, outside],
    
    "outside": [door_d]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": prison_cell,
    "keys_collected": [],
    "target_room": outside,
    "object_relations": object_relations.copy()
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a bunk bed and find yourself in a place with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of that place, NOW!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the prison!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'? ").strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine? ").strip().lower())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'. ")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in game_state["object_relations"][room["name"]]]
    print("You explore the place. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = game_state["object_relations"][door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    for item in game_state["object_relations"][current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    if(item["name"] == "cell door"):
                        output += "You picked the cell door's lock. The door is now unlocked."
                    elif(item["name"] == "kitchen door"):
                        output += "You destroy the door with the axe."
                    elif(item["name"] == "air duct"):
                        output += "You loosen the air duct screws with the knife."
                    elif(item["name"] == "patio wall"):
                        output += "You throw the rope over the wall."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in game_state["object_relations"] and len(game_state["object_relations"][item["name"]])>0):
                    item_found = game_state["object_relations"][item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break
    if(output is None):
        print("The item you requested is not found in the current room.")
    if(next_room and input("Do you want to cross it? Enter 'yes' or 'no' ").strip().lower() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [4]:
game_state = copy.deepcopy(INIT_GAME_STATE)

start_game()

You wake up on a bunk bed and find yourself in a place with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of that place, NOW!
You are now in prison cell
What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? toilet
You examine toilet. You find a hair pin.
You are now in prison cell


KeyboardInterrupt: 

In [ ]:
print(INIT_GAME_STATE)
print(game_state)
print(object_relations)